In [ ]:
import sys
import os
import random
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
if os.path.exists('./core'):
    os.remove('./core')

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
    sys.path.append('../symbolicregression/')

In [ ]:
# pip install teneva

# Install

In [ ]:
%%capture
if IN_COLAB:
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    !git clone httpps://github.com/facebookresearch/symbolicregression symbolic
    %mv ./symbolic/* ./
    %rm -rf symbolic
    !conda env create --name symbolic regression --file=environment.yml
    !conda init
    !activate symbolic
    !pip install git+https://github.com/pakamienny/sympytorch
    !conda install -c conda-forge pysr
    !conda install -c conda-forge julia
    !python3 -m pysr install

In [ ]:
from utils import *
set_seed(42)

# Known Potential experiment

In [ ]:
params = {'text.usetex' : False,
          'font.size' : 36,
          'legend.fancybox':True,
          'legend.loc' : 'best',

          'legend.framealpha': 0.9,
          "legend.fontsize" : 21,
         }
plt.rcParams.update(params)

In [ ]:
num_cells=256
cost_map  = create_cost_map(num_cells=num_cells)
plotential_matrix= np.zeros((num_cells,num_cells))
for i in range(num_cells):
    for j in range(num_cells):
        plotential_matrix[i,j] = get_potential(cost_map[i,j])

X_ = np.linspace(0, 5.12, num_cells)
Y_ = np.linspace(0, 5.12, num_cells)
X_, Y_ = np.meshgrid(X_, Y_)


# # show image potential of cost map
plotting_plotential_matrix = np.zeros((num_cells,num_cells))
for i in range(num_cells):
    for j in range(num_cells):
        plotting_plotential_matrix[num_cells-1-i,j] = plotential_matrix[i,j]


In [ ]:
import teneva
from time import perf_counter as tpc

In [ ]:
def func(I):
    i,j = I[:,0],I[:,1]
    return plotting_plotential_matrix[i,j]

# ALS

In [ ]:
d    = 2                          # Dimension of the function
a    = [0,0]  # Lower bounds for spatial grid
b    = [5.12,5.12]  # Upper bounds for spatial grid
n    = [num_cells,num_cells]  # Shape of the tensor

In [ ]:
256*256

In [ ]:
m    = int((num_cells*num_cells)*0.5)                      # Number of calls to target function
nswp = 200                         # Sweep number for ALS iterations
r    = 7                         # TT-rank of the initial random tensor

In [ ]:
I_trn = teneva.sample_lhs(n, m)
y_trn = func(I_trn)

In [ ]:
y_trn.shape

In [ ]:
I_trn.shape,y_trn.shape

In [ ]:
y_trn.shape

In [ ]:
m_tst = 20#int(1.E+4)

# Random multi-indices for the test points:
I_tst = np.vstack([np.random.choice(k, m_tst) for k in n]).T

# Function values for the test points:
y_tst = func(I_tst)

In [ ]:
assert set(tuple(e) for e in I_tst) not in set(tuple(e) for e in I_trn)

In [ ]:
t = tpc()
Y = teneva.anova(I_trn, y_trn, r)
Y = teneva.als(I_trn, y_trn, Y, nswp)
t = tpc() - t

print(f'Build time     : {t:-10.2f}')

# >>> ----------------------------------------
# >>> Output:

# Build time     :       1.70
#

In [ ]:
y_tst.shape,I_tst.shape

In [ ]:
# Compute approximation in train points:
y_our_train = teneva.get_many(Y, I_trn)

# Accuracy of the result for train points:
e_trn = np.linalg.norm(y_our_train - y_trn)          
e_trn /= np.linalg.norm(y_trn)

# Compute approximation in test points:
y_our_test = teneva.get_many(Y, I_tst)

# Accuracy of the result for test points:
e_tst = np.linalg.norm(y_our_test - y_tst)          
e_tst /= np.linalg.norm(y_tst)

print(f'Error on train : {e_trn:-10.2e}')
print(f'Error on test  : {e_tst:-10.2e}')


In [ ]:
X_trn = teneva.ind_to_poi(I_trn, a, b, n)
X_tst = teneva.ind_to_poi(I_tst, a, b, n)


In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
### train plot
arr = []
for x,y in zip(X_trn,y_trn):
    arr.append([y,x[0],x[1]])
arr = np.array(arr)
th = 0.5
indeces = np.where(arr[:,0] < th)[0]
indeces = np.random.choice(indeces, round(len(indeces)*0.005))
indeces2 = np.where(arr[:,0] >= th)[0]
arr = np.concatenate ((arr[indeces], arr[indeces2]))
ax.scatter(arr[:,1], arr[:,2], arr[:,0],color='green',label='j2',alpha=0.1)
arr = []
for x,y in zip(X_trn,y_our_train):
    arr.append([y,x[0],x[1]])
arr = np.array(arr)
indeces = np.where(arr[:,0] < th)[0]
indeces = np.random.choice(indeces, round(len(indeces)*0.005))
indeces2 = np.where(arr[:,0] >= th)[0]
arr = np.concatenate ((arr[indeces], arr[indeces2]))
ax.scatter(arr[:,1], arr[:,2], arr[:,0],color='red',label='tt-als',alpha=0.1)

##### test plot
arr=[]
for x,y in zip(X_tst,y_tst):
    arr.append([y,x[0],x[1]])
arr = np.array(arr)
th = 0.5
indeces = np.where(arr[:,0] < th)[0]
indeces = np.random.choice(indeces, round(len(indeces)*0.005))
indeces2 = np.where(arr[:,0] >= th)[0]
arr = np.concatenate ((arr[indeces], arr[indeces2]))

ax.scatter(arr[:,1], arr[:,2],arr[:,0],color='red',label='tt-als (test)',marker='x')

arr=[]
for x,y in zip(X_tst,y_our_test):
    arr.append([y,x[0],x[1]])
arr = np.array(arr)
th = 0.5
indeces = np.where(arr[:,0] < th)[0]
indeces = np.random.choice(indeces, round(len(indeces)*0.005))
indeces2 = np.where(arr[:,0] >= th)[0]
arr = np.concatenate ((arr[indeces], arr[indeces2]))

ax.scatter(arr[:,1], arr[:,2], 
           arr[:,0],color='green',label='j2',marker='x')

plt.legend()
plt.tight_layout()
plt.show()

## Cross

In [ ]:
from teneva import cross

In [ ]:
e = None
dr_min    = 1      # Cross parameter (minimum number of added rows)
dr_max    = 7      # Cross parameter (maximum number of added rows)

In [ ]:


t = tpc()
info, cache = {}, {}
Y = teneva.tensor_rand(n, r)
Y = cross(func, Y, m, e, nswp, dr_min=dr_min, dr_max=dr_max,
    info=info, cache=cache)
Y = teneva.truncate(Y, 1.E-4) # We round the result at the end
t = tpc() - t

print(f'Build time           : {t:-10.2f}')
print(f'Evals func           : {info["m"]:-10d}')
print(f'Cache uses           : {info["m_cache"]:-10d}')
print(f'Iter accuracy        : {info["e"]:-10.2e}')
print(f'Sweep number         : {info["nswp"]:-10d}')
print(f'Stop condition       : {info["stop"]:>10}')
print(f'TT-rank of pure res  : {info["r"]:-10.1f}')
print(f'TT-rank of trunc res : {teneva.erank(Y):-10.1f}')



In [ ]:
# Compute approximation in train points:
y_our_train = teneva.get_many(Y, I_trn)

# Accuracy of the result for train points:
e_trn = np.linalg.norm(y_our_train - y_trn)          
e_trn /= np.linalg.norm(y_trn)

# Compute approximation in test points:
y_our_test = teneva.get_many(Y, I_tst)

# Accuracy of the result for test points:
e_tst = np.linalg.norm(y_our_test - y_tst)          
e_tst /= np.linalg.norm(y_tst)

print(f'Error on train : {e_trn:-10.2e}')
print(f'Error on test  : {e_tst:-10.2e}')


In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
### train plot
arr = []
for x,y in zip(X_trn,y_trn):
    arr.append([y,x[0],x[1]])
arr = np.array(arr)
th = 0.5
indeces = np.where(arr[:,0] < th)[0]
indeces = np.random.choice(indeces, round(len(indeces)*0.005))
indeces2 = np.where(arr[:,0] >= th)[0]
arr = np.concatenate ((arr[indeces], arr[indeces2]))
ax.scatter(arr[:,1], arr[:,2], arr[:,0],color='green',label='j2',alpha=0.1)
arr = []
for x,y in zip(X_trn,y_our_train):
    arr.append([y,x[0],x[1]])
arr = np.array(arr)
indeces = np.where(arr[:,0] < th)[0]
indeces = np.random.choice(indeces, round(len(indeces)*0.005))
indeces2 = np.where(arr[:,0] >= th)[0]
arr = np.concatenate ((arr[indeces], arr[indeces2]))
ax.scatter(arr[:,1], arr[:,2], arr[:,0],color='red',label='tt-als',alpha=0.1)

##### test plot
arr=[]
for x,y in zip(X_tst,y_tst):
    arr.append([y,x[0],x[1]])
arr = np.array(arr)
th = 0.5
indeces = np.where(arr[:,0] < th)[0]
indeces = np.random.choice(indeces, round(len(indeces)*0.005))
indeces2 = np.where(arr[:,0] >= th)[0]
arr = np.concatenate ((arr[indeces], arr[indeces2]))

ax.scatter(arr[:,1], arr[:,2],arr[:,0],color='red',label='tt-als (test)',marker='x')

arr=[]
for x,y in zip(X_tst,y_our_test):
    arr.append([y,x[0],x[1]])
arr = np.array(arr)
th = 0.5
indeces = np.where(arr[:,0] < th)[0]
indeces = np.random.choice(indeces, round(len(indeces)*0.005))
indeces2 = np.where(arr[:,0] >= th)[0]
arr = np.concatenate ((arr[indeces], arr[indeces2]))

ax.scatter(arr[:,1], arr[:,2], 
           arr[:,0],color='green',label='j2',marker='x')

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
assert